This notebook will be used for IBM Data Science Capstone Project

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

### Get Data

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Cleaning

In [3]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [4]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

In [5]:
canada_df['Neighbourhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Queen's Park,Queen's Park


In [6]:
canada_df.shape

(103, 3)

## Get Lat and Long for Postal Codes (Part 2)

In [ ]:
import geocoder

post_df = canada_df['Postcode'].tolist()

for postal_code in post_df:
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        print(g)
        lat_lng_coords = g.latlng
        
        #print(lat_lng_coords)
        
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    canada_df.loc[canada_df.Postcode == postal_code, 'Latitude'] = latitude
    canada_df.loc[canada_df.Postcode == postal_code, 'Longitude'] = longitude
    

canada_df.head()
    

Geocoder was unreliable therefore below is import of the CSV

In [14]:
post_codes = pd.read_csv("https://cocl.us/Geospatial_data")
post_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
#print(post_df)
for postal_code in post_df:
    lat = post_codes.loc[post_codes['Postal Code'] == postal_code, 'Latitude']
    long = post_codes.loc[post_codes['Postal Code'] == postal_code, 'Longitude']
    mask = canada_df['Postcode'] == postal_code
    canada_df.loc[mask, 'Latitude'] = float(lat)
    canada_df.loc[mask, 'Longitude'] = float(long)
    
    #print(post_codes.loc[post_codes['Postal Code'] == postal_code, 'Latitude'])
    
    #print(canada_df.loc[mask, 'Latitude'] )
    #print(lat)
    
canada_df.head()




    
    

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
7,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [30]:
canada_df.reset_index(drop=True, inplace=True)

In [31]:
canada_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
